# 1 - Instalações, imports

## 1.0 - Instalação da biblioteca Surprise

In [ ]:
pip install surprise

## 1.1 - Import das bibliotecas

In [2]:
import pandas as pd

from surprise                                     import Reader, Dataset
from surprise.prediction_algorithms.knns          import KNNBaseline
from surprise.model_selection                     import train_test_split
from surprise.prediction_algorithms.slope_one     import SlopeOne
from surprise.prediction_algorithms.co_clustering import CoClustering
from surprise                                     import accuracy

### 1.2 - Import dos databases

In [3]:
recipes = pd.read_csv('RAW_recipes.csv')
interactions = pd.read_csv('RAW_interactions.csv')

# 2 - Tratamento das bases

## 2.1 - Limpeza e visualização do Dataset

In [4]:
# Juntando as bases
receitas = interactions.join(recipes.set_index('id'), on='recipe_id')

# Removendo algumas colunas que não serão necessárias
colunas_para_remover = ['review', 'nutrition', 'steps', 'ingredients', 'submitted']
receitas_limpa = receitas.drop(columns=colunas_para_remover)
receitas_limpa.head()

,user_id,recipe_id,date,rating,name,minutes,contributor_id,tags,n_steps,description,n_ingredients
0,38094,40893,2003-02-17,4,white bean green chile pepper soup,495,1533,"['weeknight', 'time-to-make', 'course', 'main-...",4,easy soup for the crockpot.,9
1,1293707,40893,2011-12-21,5,white bean green chile pepper soup,495,1533,"['weeknight', 'time-to-make', 'course', 'main-...",4,easy soup for the crockpot.,9
2,8937,44394,2002-12-01,4,devilicious cookie cake delights,20,56824,"['30-minutes-or-less', 'time-to-make', 'course...",5,NaN,4
3,126440,85009,2010-02-27,5,baked potato toppings,10,64342,"['15-minutes-or-less', 'time-to-make', 'course...",3,these toppings sure makes a nice change from p...,13
4,57222,85009,2011-10-01,5,baked potato toppings,10,64342,"['15-minutes-or-less', 'time-to-make', 'course...",3,these toppings sure makes a nice change from p...,13


In [29]:
# Devido ao tamanho do dataframe se faz necessário reduzir o mesmo para não gerar estouro de memória
frac = 0.09  # proporção da amostra (10%)
receitas_amostradas = receitas_limpa.sample(frac=frac, random_state=42)  # seleção dos dados de forma aleatória

In [30]:
# Fazendo uma breve análise
print('Quantidade de receitas: ',
       receitas_amostradas['recipe_id'].value_counts().shape[0] )

print('Quantidade de usuários avaliando: ',
       receitas_amostradas['user_id'].value_counts().shape[0] )

print('Quantidade de Avaliações: ',
       receitas_amostradas.value_counts().shape[0] )

Quantidade de receitas:  58505
Quantidade de usuários avaliando:  38162
Quantidade de Avaliações:  99797


In [31]:
# Receitas mais bem avaliadas
melhores_receitas = receitas_amostradas.sort_values(by='rating', ascending=False).head(10)
melhores_receitas[['recipe_id', 'name', 'rating']]

,recipe_id,name,rating
592612,32614,fudge crinkles a great 4 ingredient cake mix ...,5
676100,27630,shortbread fruit tartlets,5
933645,11150,cherry angel,5
372723,301380,cinnamon swirl muffins,5
764877,81289,chocolate lemonade,5
897489,62524,pork with pineapple,5
478652,43509,crumb topped banana muffins,5
925448,25625,now don t go eeewww before you try this,5
494679,208414,barefoot contessa s blueberry crumb cake,5
1059141,199527,fast microwave fudge nut free,5


# 3.0 - Modelos e Treinamentos - Surprise

## 3.1 - Divisão dos dados para treinamento

In [32]:
# Configuração para treinamento
reader = Reader(rating_scale=(0,5))
# Seleção das variáveis para o modelo
data = Dataset.load_from_df(receitas_amostradas[['user_id', 'recipe_id', 'rating']], reader)
# Divisão dos dados de treino e teste
trainset, testset = train_test_split(data, test_size=.25, random_state=42)
# Configurações das medidas de similaridade
sim_options = { 'name': 'pearson_baseline', 'user_based': True }

## 3.2 - Treinamento do Modelo

#### Faremos aqui treinamento em 3 tipos de modelo para analisar qual terá a melhor acurácia para podermos utilizar

In [33]:
# Criação do modelo
knn = KNNBaseline(k=33, sim_options=sim_options)
# Treinamento do modelo
knn.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [34]:
# Criação do modelo
slo = SlopeOne()
# Treinamento do modelo
slo.fit(trainset)

In [35]:
# Criação do modelo
co = CoClustering(n_epochs=10, verbose=True, random_state=42)
# Treinamento do modelo
co.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


## 3.3 - Avaliar a acurácia dos modelos

In [40]:
# Acurácia KNNBaseline
predictions_knn = knn.test(testset)
accuracy.rmse(predictions_knn)

RMSE: 1.2715


1.2714693201470446

In [41]:
# Acurácia SlopeOne
predictions_slo = slo.test(testset)
accuracy.rmse(predictions_slo)

RMSE: 1.3228


1.3227736569478397

In [42]:
# Acurácia CoClustering
predictions_co = co.test(testset)
accuracy.rmse(predictions_co)

RMSE: 1.3832


1.3831713599422963

# 4.0 - Função para gerar as melhores recomendações

#### Conforme é possível verificar nos códigos acima o algoritmo que obteve a melhor performance foi o KNN, iremos utilizar ele

In [46]:
def top_n(user_id,n):
  # Selecionando apenas as receitas do treinamento
  lista_receitas_treino = []
  for x in trainset.all_items():
    lista_receitas_treino.append(trainset.to_raw_iid(x))
  # Selecionando as receitas do treinamento que o usuário não avaliou
  receitas_user = receitas_amostradas.query('user_id == @user_id')['recipe_id'].values
  receitas_user_nao = receitas_amostradas.query('recipe_id not in @receitas_user')
  receitas_user_nao = receitas_user_nao.query('recipe_id in @lista_receitas_treino')['recipe_id'].values
  # Criando um ranking para o usuário para as receitas não avaliados
  ranking=[]
  for recipe_id in receitas_user_nao:
    ranking.append((recipe_id, knn.predict(trainset.to_inner_uid(user_id), trainset.to_inner_iid(recipe_id))[3]))
  # Ordenando as TOP receitas avaliados
  ranking.sort(key=lambda x: x[1], reverse=True)
  # Selecionando os Ids das receitas
  x,_ = zip(*ranking[:n])
  # Listando os nomes das receitas em ordem de recomendação
  return receitas_amostradas.query('recipe_id in @x')['name'].copy().reset_index(drop=True)

In [44]:
receitas_amostradas.head()

,user_id,recipe_id,date,rating,name,minutes,contributor_id,tags,n_steps,description,n_ingredients
592612,764278,32614,2012-12-22,5,fudge crinkles a great 4 ingredient cake mix ...,15,37305,"['15-minutes-or-less', 'time-to-make', 'course...",20,"these are chewy, fudgy, super easy cookies tha...",4
613724,51546,57355,2008-12-27,5,mashed baked potatoes,60,73242,"['60-minutes-or-less', 'time-to-make', 'main-i...",6,i got this recipe from my sister who is a grea...,6
877117,133174,241394,2007-08-06,4,black and red mexican slow cooker soup,500,305147,"['course', 'main-ingredient', 'cuisine', 'prep...",4,this is a staple in our house. serve it with ...,11
269484,1134625,169227,2010-05-11,5,chicken barley casserole,75,216999,"['time-to-make', 'course', 'main-ingredient', ...",18,this is one of my family's favorite recipes. ...,13
366684,51011,121093,2006-09-26,5,mushroom sauce for meatballs,20,59780,"['30-minutes-or-less', 'time-to-make', 'course...",8,adapted from company's coming make ahead meals...,11


In [48]:
# Chamando a função
id_input = input("Informe o ID do usuário para indicar as 5 melhores receitas: ")

top_n(764278, 5) # passando o id do usuário e a quantidade de receitas a ser exibida

0    salmon with balsamic vinegar sauce
1    salmon with balsamic vinegar sauce
2     stir fried shrimp in garlic sauce
3     stir fried shrimp in garlic sauce
4     stir fried shrimp in garlic sauce
Name: name, dtype: object